In [1]:
import pandas as pd
flights_df = pd.read_csv('D:\\Assignment\\flights.csv')

In [2]:
flights_df = flights_df.drop(['DepTime', 'ArrTime', 'DepDel15', 'ArrDel15'], axis = 1)

In [3]:
flights_df['DayOfWeek'] = pd.to_datetime(flights_df['FlightDate']).dt.dayofweek


In [4]:
flights_df['DayOfWeek'].unique()

array([4, 5, 6, 0, 1, 2, 3])

In [5]:
flights_df = flights_df.drop('FlightDate', axis = 1)

In [6]:
import gc
gc.collect()


0

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(flights_df['Origin'])


LabelEncoder()

In [8]:
flights_df['Origin'] = le.transform(flights_df['Origin'])
flights_df['Dest'] = le.transform(flights_df['Dest'])

In [9]:
flights_df['Origin'].unique()

array([13,  6, 12,  9,  2, 10,  3,  5,  8,  7, 14, 11,  0,  1,  4])

In [10]:
import numpy as np

flights_df['Delay'] = np.where((flights_df['ArrDelayMinutes'] > 0.0) | (flights_df['DepDelayMinutes'] > 0.0), 1, 0)
flights_df = flights_df.drop(['DepDelayMinutes', 'ArrDelayMinutes'], axis = 1)


In [11]:
X = flights_df.iloc[:, :-1].values
y = flights_df.iloc[:, -1].values

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
X_train.shape

(1423917, 9)

In [14]:
flights_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'Origin', 'Dest',
       'CRSDepTime', 'CRSArrTime', 'DayOfWeek', 'Delay'],
      dtype='object')

In [15]:
X_train[0]

array([2016,    4,   12,   15,   12,    0,  700, 1228,    3], dtype=int64)

In [16]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [17]:
gc.collect()

0

In [18]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier

classifiers = [
    ('Decision Tree', DecisionTreeClassifier(random_state = 0)),
    ('XGBoost', XGBClassifier(objective='binary:logistic', random_state=3)),
    ('extra_trees_classifier' ,ExtraTreesClassifier(n_estimators=100, random_state=0))
]

results = {}


In [19]:
from sklearn.metrics import accuracy_score
from dask_ml.wrappers import ParallelPostFit
from sklearn.pipeline import make_pipeline
import dask
from sklearn.metrics import confusion_matrix


for clf_name, classifier in classifiers:
    pipeline = make_pipeline(classifier)
    with dask.config.set(scheduler='threads'): 
        pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    results[clf_name] = {'accuracy': accuracy, 'confusion_matrix': conf_matrix}
    print(f'{clf_name} Accuracy: {accuracy:.4f}\n')
    print(f'{clf_name} Confusion Matrix:\n{conf_matrix}\n')
    gc.collect()


Decision Tree Accuracy: 0.8941

Decision Tree Confusion Matrix:
[[213021  25003]
 [ 25245 211371]]

XGBoost Accuracy: 0.6404

XGBoost Confusion Matrix:
[[154336  83688]
 [ 87000 149616]]



MemoryError: could not allocate 134217728 bytes

In [20]:
# A memory error